<a href="https://colab.research.google.com/github/gaonkarrs/AIML/blob/main/CapstoneRagBench0531_V02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
import torch

# Step 1: Load the dataset
dataset = load_dataset("rungalileo/ragbench", "pubmedqa", split="test")

# Step 2: Flatten and collect document text
all_docs = []
for item in dataset:
    all_docs.extend(item["documents"])  # each item["documents"] is a list

# Optional: Remove duplicates
all_docs = list(set(all_docs))

# Step 3: Create chunks (customize chunk size and overlap)
def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if chunk:
            chunks.append(chunk)
    return chunks

chunked_docs = []
for doc in all_docs:
    chunked_docs.extend(chunk_text(doc))

# ✅ Optional: Limit for quick testing (comment out for full dataset)
MAX_CHUNKS = 1000
chunked_docs = chunked_docs[:MAX_CHUNKS]

# Step 4: Embed with SentenceTransformers (with GPU and batching)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)

document_embeddings = embedder.encode(
    chunked_docs,
    batch_size=32,  # Tune based on memory
    convert_to_numpy=True,
    show_progress_bar=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/80.1M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/10.3M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [4]:
dataset[0]['question']

'Do Surface Porosity and Pore Size Influence Mechanical Properties and Cellular Response to PEEK?'

In [5]:
import faiss
import numpy as np
import os
from datasets import Dataset

# Step 5: Create HF dataset
hf_dataset = Dataset.from_dict({"text": chunked_docs})

# Step 6: Create and save FAISS index manually
# Create FAISS index
dim = document_embeddings[0].shape[0]
faiss_index = faiss.IndexFlatL2(dim)
faiss_index.add(np.array(document_embeddings).astype('float32'))

# Create paths
dataset_path = "rag_pubmedqa_dataset"
index_path = "rag_pubmedqa_index/faiss.index"

# Ensure directories exist
os.makedirs(dataset_path, exist_ok=True)
os.makedirs(os.path.dirname(index_path), exist_ok=True)

# Save dataset and FAISS index separately
hf_dataset.save_to_disk(dataset_path)
faiss.write_index(faiss_index, index_path)

print(f"✅ Dataset saved to: {dataset_path}")
print(f"✅ FAISS index saved to: {index_path}")


Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset saved to: rag_pubmedqa_dataset
✅ FAISS index saved to: rag_pubmedqa_index/faiss.index


In [6]:
# Example retrieval function (you already have this):
def retrieve_top_k(query, embedder, k=5):
    query_embedding = embedder.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = faiss_index.search(query_embedding, k)
    results = [hf_dataset[int(i)]["text"] for i in indices[0]]
    return results

In [7]:
def generate_answer_together(context, question, max_tokens=200):
    # Build prompt for Llama with retrieved context and question
    prompt = (
        "You are a helpful biomedical assistant. Use the following context from PubMed articles to answer the question.\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {question}\n"
        "Answer:"
    )

    payload = {
        "model": "meta-llama/Llama-3.3-70B-Instruct-Turbo",
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "parameters": {
            "max_new_tokens": max_tokens,
            "temperature": 0.7,
            "top_p": 0.9,
            "stop": ["\n\n"]
        }
    }

    response = requests.post(API_URL, headers=headers, json=payload)

    if response.status_code == 200:
        data = response.json()
        # The response structure for Together.ai's llama model uses choices[0].text for output
        answer = data.get("output", {}).get("choices", [{}])[0].get("text", "").strip()
        return answer
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

In [8]:
import requests

# Your Together API key (replace with your own)
API_KEY = "c1531e2c6b2242fe757c19d3f2a82d076e2b085059a9487c8e999c4ef01113aa"

# Together.ai inference API URL for meta-llama/Llama-3.3-70B-Instruct-Turbo model
API_URL = "https://api.together.xyz/inference"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Usage example
query = dataset[5]['question']
top_chunks = retrieve_top_k(query, embedder, k=5)
context = "\n\n".join(top_chunks)

#print("Context:\n", context)
print("\nQuestion:\n", query)
answer = generate_answer_together(context, query)
print("Answer:", answer)



Question:
 Are prostate biopsies mandatory in patients with prostate-specific antigen increase during intravesical immuno- or chemotherapy for superficial bladder cancer?
Answer: No, prostate biopsies are not mandatory in patients with prostate-specific antigen (PSA) increase during intravesical immuno- or chemotherapy for superficial bladder cancer. The context suggests that the interpretation of total PSA can be problematic in this population, and the decision to perform prostatic biopsies should be based on individual patient factors and clinical judgment, rather than a mandatory requirement for all patients with a PSA increase.
